<a href="https://colab.research.google.com/github/amittal27/course-v3/blob/master/planet_experimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

### **Configure data**

In [3]:
path = Path.cwd()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/content/planet')

### **Multiclassification**

In [4]:
# read the csv file using the pandas library (popular way of dealing with tabular data in python), print the first five rows
df = pd.read_csv(path/'train_v2.csv')
df.head()

FileNotFoundError: ignored

In [5]:
# data augmentation
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0) # do not want warp on the satellite images

In [6]:
# ensure we have the same validation set each time
np.random.seed(42)
# get images, get labels
src = (ImageList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .split_by_rand_pct(0.2) # set aside 20% of the training set (the current set of images) for the validation set
       .label_from_df(label_delim=' '))

FileNotFoundError: ignored

In [ ]:
# apply transforms, construct dataset
data = (src.transform(tfms, size=128) # standardize images to be a little smaller, 128 x 128
        .databunch().normalize(imagenet_stats)) # use databunch to bind training and validation datasets

FileNotFoundError: ignored

In [7]:
data.show_batch(rows=3, figsize=(12,9))

NameError: ignored

In [ ]:
# base artchitecture
arch = models.resnet50

### **Metrics**
metrics to print out during training (NOTE: they do not impact how our model trains); just shows us how we're doing

In [ ]:
# however, instead of picking just one of the classes in len(data.classes) as our prediction label, we want to pick out n of those classes
# anything higher than a desired threshold will be assumed to be a label for the input image
# accuracy uses argmax to find the category with the maximum probability of being represented by the image/data given
# it compared it to the actual accuracy and took the average; this method can't be implemented when we have multiple labels per image
# in this case, our threshold value is 0.2 (experimentally found to be pretty good)

# a partial function takes in a function and a list of keywords & values and creates a new func that's exactly the same but with the arguments provided; new function was generated
acc_02 = partial(accuracy_thresh, thresh=0.2) 
data.c # number of outputs we want our model to create = len(data.classes); given one probability for each of these classes

In [ ]:
f_score = partial(fbeta, thresh=0.2) # a metric used by Kaggle to weigh false positive and false negatives
learn = cnn_learner(data, arch, metrics=[acc_02, f_score])

In [ ]:
# find a good learning rate
learn.lr_find()

In [ ]:
# plot results
learn.recorder.plot()

In [ ]:
# pick learning rate
lr = 1e-2

In [ ]:
# fit_one_cycle five times with that learning rate
learn.fit_one_cycle(5, slice(lr))

In [ ]:
# save
learn.save('stage-1-rn50')

In [14]:
learn.unfreeze()

NameError: ignored

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
# fitting with original dataset, though we could create a new databunch with just the misclassified instances
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-rn50')

### **Now, create a whole "new" dataset where the images are 256 x 256 to hopefully increase our fbeta metric score**
no concern of overfitting then

In [ ]:
# create new databunch with 256 x 256 images (higher res images)
data = (src.transform(tfms, size=256), # same transforms as before
        .databunch.normalize(imagenet_stats))

In [15]:
# start with our pre-trained model
learn.data = data # replace learner data with new databunch
data.train_ds[0][0].shape

NameError: ignored

In [ ]:
# freeze to just train the last few layers
learn.lr_find()
learn.recorder.plot()

In [ ]:
# new learning rate
lr=1e-2/2

In [ ]:
# just training the last few layers
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-256-rn50')

In [ ]:
learn.export()